In [ ]:
import sklearn as sk
import streamlit as st
import pandas as pd
import numpy as np
import sklearn as sk
import xgboost as xgb
import tensorflow as tf
import keras as ks
import seaborn as sns
import plotly.express as px
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

filepath = "https://raw.githubusercontent.com/lizh1988/portfolio/main/Conc%20strength%20predictor/concrete_data.csv"

df = pd.read_csv(filepath, sep = ',')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns={'blast_furnace_slag':'slag',
         'fly_ash':'ash', 
         'superplasticizer':'sp',
         'coarse_aggregate':'coarse',
         'concrete_compressive_strength':'strength',
         'fine_aggregate ':'fine'}, inplace=True)
df.head()

In [ ]:
fig = sns.boxplot(data=df)
fig

In [ ]:
scaler=sk.preprocessing.StandardScaler()

x = df.drop(['strength'],axis=1)

x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(x ,df['strength'], test_size=0.2, random_state=88)

#x_train=scaler.fit_transform(x_train)
#x_test=scaler.transform(x_test)

In [ ]:
#le = sk.preprocessing.LabelEncoder()
#y_train=le.fit_transform(y_train)
#y_test=le.transform(y_test)

#xgb.cv is not used due to excessive runtime

reg=xgb.XGBRegressor()
reg.fit(x_train,y_train)

In [ ]:
kfold=sk.model_selection.KFold(n_splits=10, shuffle=True)

score=sk.model_selection.cross_val_score(reg, x_train, y_train,cv=kfold)
score.mean()

In [ ]:
ypred=reg.predict(x_test)
mse=sk.metrics.mean_squared_error(y_test,ypred)
r2=sk.metrics.r2_score(y_test,ypred)
r2

In [14]:
#parameters = {'device':['cuda'],
#                'nthread':[2], #when use hyperthread, xgboost may become slower
#              'objective':['reg:linear'],
#              'learning_rate': [.03, 0.05, .07], #so called `eta` value
#              'max_depth': [5, 6, 7,8,9],
#              'min_child_weight': [4],
#              'verbosity': [1],
#              'subsample': [0.5, 0.7, 1],
#              'colsample_bytree': [1],
#              'n_estimators': [500]}

space = {
        'device':'cuda',
        'max_depth':hp.choice('max_depth', np.arange(4, 25, 1, dtype=int)),
        'n_estimators':hp.choice('n_estimators', np.arange(100, 10000, 10, dtype=int)),
        'colsample_bytree':hp.quniform('colsample_bytree', 0.5 ,0.9, 0.1),
        'min_child_weight':hp.choice('min_child_weight', np.arange(50, 350, 10, dtype=int)),
        'subsample':hp.quniform('subsample', 0.7, 0.9, 0.1),
        'learning_rate':hp.quniform('eta', 0.05, 0.3, 0.05),
        
        'objective':'reg:squarederror',
        
        'verbosity':0,

        'tree_method':'gpu_hist',
        'eval_metric': 'rmse',
    }

def score(params):
    model = xgb.XGBRegressor(**params)
    
    model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)],
              verbose=False, early_stopping_rounds=10)
    Y_pred = model.predict(x_test).clip(0, 20)
    score = np.sqrt(sk.metrics.mean_squared_error(y_test, Y_pred))
    print(score)
    return {'loss': score, 'status': STATUS_OK}    
    
def optimize(trials, space):
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=2000)
    return best

trials = Trials()
best_params = optimize(trials, space)

# Return the best parameters
print(space_eval(space, best_params))



#reg_grid=sk.model_selection.GridSearchCV(reg, parameters, scoring='r2', n_jobs=None, refit=True, cv=None, verbose=2, return_train_score=False)

#y_pred1=reg_grid.fit(x_train,y_train)

24.42301418407412                                                                   
 84%|████████▍ | 1677/2000 [13:55<02:29,  2.16trial/s, best loss: 24.20638337230211]

c:\Users\lizhi\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



24.322698052846505                                                                  
 84%|████████▍ | 1678/2000 [13:55<02:16,  2.37trial/s, best loss: 24.20638337230211]

c:\Users\lizhi\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



24.387395016043378                                                                  
 84%|████████▍ | 1679/2000 [13:56<02:03,  2.60trial/s, best loss: 24.20638337230211]

...

c:\Users\lizhi\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



24.42029083301286                                                                   
100%|█████████▉| 1998/2000 [17:02<00:01,  1.57trial/s, best loss: 24.20638337230211]

c:\Users\lizhi\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

c:\Users\lizhi\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



24.322698052846505                                                                  
24.42029083301286                                                                   
100%|██████████| 2000/2000 [17:03<00:00,  1.95trial/s, best loss: 24.20638337230211]
{'colsample_bytree': 0.9, 'eval_metric': 'rmse', 'gpu_id': 0, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 70, 'n_estimators': 170, 'objective': 'reg:squarederror', 'subsample': 0.8, 'tree_method': 'gpu_hist', 'verbosity': 0}


c:\Users\lizhi\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



In [ ]:
#score=sk.model_selection.cross_val_score(reg_grid, x_train, y_train,cv=kfold)
#score.mean()

optparam=(space_eval(space, best_params))
#for i in space_eval(space, best_params):
#    print (k)
regopt=xgb.XGBRegressor(**optparam)
print(regopt)
#print(space)
#print(**best_params)
regopt.fit(x_train,y_train)
ypred=regopt.predict(x_test)
mse=sk.metrics.mean_squared_error(y_test,ypred)
r2=sk.metrics.r2_score(y_test,ypred)
print(r2)

In [ ]:
mse